In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split



In [2]:
author_title = pd.read_csv('../data/author_title.csv')
print(author_title.shape)
author_title.head(4)


(7531, 2)


,Author,Title
0,"Color Photography, Vol. 1, No. 1 Various","Birds, Illustrated"
1,Édouard Cuyer,Artistic Anatomy of Animals
2,Frank M. Chapman,What Bird is That?
3,A. Mueller,On Snake-Poison: Its Action and Its Antidote


In [3]:
prompts = pd.read_csv('../data/rich_book_prompts.csv')
print(prompts.shape)    
prompts.head(4)

(64552, 3)


,Author,Title,Prompt
0,H. Beam Piper,He Walked Around the Horses,"Write a piece in the style of H. Beam Piper, r..."
1,H. Beam Piper,He Walked Around the Horses,Describe a fictional situation that embodies t...
2,H. Beam Piper,He Walked Around the Horses,Write about a character who suddenly finds the...
3,H. Beam Piper,He Walked Around the Horses,"Craft a narrative where an ancient secret, bur..."


In [4]:
train, test = train_test_split(prompts, test_size=0.2, random_state=42)
print(train.shape)
print(test.shape)

(51641, 3)
(12911, 3)


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(prompts['Author'])
prompts['label'] = le.transform(prompts['Author'])
prompts.head(4)

,Author,Title,Prompt,label
0,H. Beam Piper,He Walked Around the Horses,"Write a piece in the style of H. Beam Piper, r...",1497
1,H. Beam Piper,He Walked Around the Horses,Describe a fictional situation that embodies t...,1497
2,H. Beam Piper,He Walked Around the Horses,Write about a character who suddenly finds the...,1497
3,H. Beam Piper,He Walked Around the Horses,"Craft a narrative where an ancient secret, bur...",1497


In [6]:
from datasets import Dataset

dataset_ = Dataset.from_pandas(prompts[['Prompt', 'label']])
dataset_ = dataset_.train_test_split(test_size=0.2)
dataset_

DatasetDict({
    train: Dataset({
        features: ['Prompt', 'label'],
        num_rows: 51641
    })
    test: Dataset({
        features: ['Prompt', 'label'],
        num_rows: 12911
    })
})

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"  # Or try others like "bert-base-uncased", "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(example):
    return tokenizer(example["Prompt"], padding="max_length", truncation=True)

tokenized_dataset = dataset_.map(tokenize_function, batched=True)


import error: No module named 'triton'


Map:   0%|          | 0/51641 [00:00<?, ? examples/s]

Map:   0%|          | 0/12911 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Prompt', 'label', 'input_ids', 'attention_mask'],
        num_rows: 51641
    })
    test: Dataset({
        features: ['Prompt', 'label', 'input_ids', 'attention_mask'],
        num_rows: 12911
    })
})

In [9]:
from transformers import AutoModelForSequenceClassification

num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none",
    use_mps_device=True ,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)
trainer.train()


/opt/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:2262: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/var/folders/4j/b7q3_s_d15v7rtfmjk70r1wh0000gn/T/ipykernel_6388/3404154946.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 